## Build a Wflow Sediment model

This notebook demonstrates how to prepare a **Wflow Sediment** model using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### HydroMT CLI build interface

Lets first check if the Wflow sediment model is recognized by HydroMT

In [ ]:
# this should return "wflow, wflow_sediment"
# as well as hydromt generic models "grid_model, lumped_model, network_model (mesh_model)"
!hydromt --models

Using the **HydroMT build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [ ]:
!hydromt build --help

### Model setup configuration

The HydroMT configuration file (YAML) contains the model setup configuration and determines which methods are used to prepare the different components of a Wflow Sediment model and in which order and optionally sets non-default arguments for each method. This configuration is passed to HydroMT using `-i <path_to_config_file>`. We have prepared several example files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_wflow/tree/main/examples) and from the [docs (building a model)](https://deltares.github.io/hydromt_wflow/latest/user_guide/sediment_build.html). 

Each header (without indent) (e.g. `setup_basemaps:`) corresponds to a model method. All model methods are explained in the [docs (model methods)](https://deltares.github.io/hydromt_wflow/latest/user_guide/sediment_model_setup.html). 

We will load the default wflow sediment build configuration yaml file for inspection:

In [ ]:
fn_config = "wflow_sediment_build.yml"
with open(fn_config, "r") as f:
    txt = f.read()
print(txt)

### Setup Wflow Sediment model from scratch

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
# NOTE: as of hydromt version 0.7.0 the region argument is optional and should be preceded by -r or --region
# NOTE: the resolution argument has been removed since hydromt v0.7.0 and can now only be configured from the configuration file under the setup_basemap header
!hydromt build wflow_sediment "./wflow_test_sediment" -r "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}" -i wflow_sediment_build.yml -d artifact_data -vv

The example above means the following: run **hydromt build** with:

* `wflow_sediment` : i.e. build a wflow sediment model
* `./wflow_test_sediment` : output model folder
* `-r "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}"` : derive a subbasin with its outlet at the given x,y coordinates and within the given bounds [xmin, ymin, xmax, ymax] (WGS84) snapped to a river with minimum stream order (strord) of 4. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/model_region)
* `-i wflow_sediment_build.yml` : model setup configuration file
* `-d artifact_data` : use pre-defined artifact_data catalog.
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.


Next we check which files have been created. The model root should contain two netcdf (.nc) files, one for the staticmaps and one for the forcing data, a wflow run configuration (.toml) file. These files are sufficient to run the wflow sediment model. In addition several geometry (.geojson) files are saved in the *staticgeoms* folder.  Finally, the setup process is logged in the hydromt.log file. 

In [ ]:
import os

root = "wflow_test_sediment"
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith(".xml"):
            continue
        print(f" - {name}")

### Visualize and/or inspect model schematization

You can (copy and) adapt the following example notebooks in order to visualize your Wflow Sediment model: 

* The **wflow plot** example notebook contains scripts to visualize your model
* The **wflow nc to raster files** example notebook contains scripts to write the nc model files to a netcdf which can be used to inspect (and modify) the model layers in e.g. QGIS.

### Add Wflow Sediment layers to an existing Wflow model

If you already have a hydrological **Wflow** model and you want to extend it in order to include **sediment** as well, then you do not need to build the wflow_sediment from scratch. You can instead **update** the wflow model with the additional components needed by wflow_sediment. 

These components are available in the template **wflow_extend_sediment.yml**:

In [ ]:
fn_config = "wflow_extend_sediment.yml"
with open(fn_config, "r") as f:
    txt = f.read()
print(txt)

Note that because we are starting from a sbm model, we start here with a ``read_config`` step in order to get a default **wflow_sediment.toml** that contains the minimum config options of the sediment model including variables names of the already present maps (eg slope, river mask...). You may have to modify this template if the staticmaps name do not correspond to the names in your own wflow sbm model.

Let's update the hydrological **Wflow** model available from our example for the Piave subbasin:

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt update wflow_sediment "wflow_piave_subbasin" -o "./wflow_test_extend_sediment" -i wflow_extend_sediment.yml -d artifact_data -vv

The example above means the following: run **hydromt update** with:

* `wflow_sediment` : i.e. update a wflow_sediment model (in our case a wflow model but with wflow_sediment components)
* `wflow_piave_subbasin` : hydrological wflow model folder
* `-o "./wflow_test_extend_sediment"`: output combined wflow hydrology+sediment models
* `-i wflow_extend_sediment.yml` : setup configuration file containing wflow sediment specific components
* `-d artifact_data` : use the pre-defined artifact_data catalog
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.